In [59]:
import sys

model_dir = '/Users/anastasiiashcherbakova/git_projects/masters_project/plasmids/2_models'
sys.path.append(model_dir)

from GANs_model_1 import *
import torch
import numpy as np
from torch.utils.data import DataLoader

In [32]:
max_length = 6000
vocab_size = 4
num_epochs = 100
noise_dim = 100
input_size = max_length * vocab_size
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

generator = Generator(noise_dim, max_length, vocab_size).to(device)
critic = Critic(input_size).to(device)  

In [33]:
print(f"Type of critic before loading state dict: {type(critic)}")

Type of critic before loading state dict: <class 'GANs_model_1.Critic'>


In [34]:
generator.load_state_dict(torch.load('/Users/anastasiiashcherbakova/Desktop/GANs_model_1_output/saved_base_generator_mod.pt', map_location=torch.device('cpu')))
critic.load_state_dict(torch.load('/Users/anastasiiashcherbakova/Desktop/GANs_model_1_output/saved_base_critic_mod.pt', map_location=torch.device('cpu')))

<All keys matched successfully>

In [35]:
generator.eval()
critic.eval()

Critic(
  (model): Sequential(
    (0): Linear(in_features=24000, out_features=128, bias=True)
    (1): LeakyReLU(negative_slope=0.2)
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=128, out_features=256, bias=True)
    (4): LeakyReLU(negative_slope=0.2)
    (5): Linear(in_features=256, out_features=128, bias=True)
    (6): LeakyReLU(negative_slope=0.2)
    (7): Dropout(p=0.3, inplace=False)
    (8): Linear(in_features=128, out_features=1, bias=True)
  )
)

In [38]:
first_linear_layer = generator.model[0]

latent_dim = first_linear_layer.weight.shape[1]
num_samples = 10

random_latent_vectors = torch.randn(num_samples, latent_dim)

with torch.no_grad():
    generated_data = generator(random_latent_vectors)

generated_data = generated_data.cpu().numpy()

# for i, sequence in enumerate(generated_data):
#     print(f"Sequence {i+1}: {sequence}")

In [44]:
generated_data.shape

(10, 6000, 4)

In [45]:
generated_data_tensor = torch.tensor(generated_data, dtype=torch.float32).to(device)


In [46]:
generated_data_tensor.shape

torch.Size([10, 6000, 4])

In [47]:
generated_data_tensor = generated_data_tensor.view(generated_data_tensor.size(0), -1)


In [48]:
generated_data_tensor.shape

torch.Size([10, 24000])

In [54]:
sequences = np.load('/Users/anastasiiashcherbakova/Desktop/cleaned_sequences.npy', allow_pickle=True)

In [55]:
base_tokenizer = Base_Level_Tokenizer()
base_vocab = base_tokenizer.fit_on_texts(sequences)

In [57]:
batch_size = 4
base_sequence_encoded = base_tokenizer.sequence_to_indices(sequences)

In [60]:
base_dataset = Dataset_Prep(base_sequence_encoded, max_length)
base_dataloader = DataLoader(base_dataset, batch_size=batch_size, shuffle=True)

In [87]:
real_data_batch = next(iter(base_dataloader)) 
real_data_batch = real_data_batch.to(device) 

In [88]:
real_data_batch

tensor([[0, 2, 2,  ..., 0, 0, 0],
        [0, 3, 3,  ..., 0, 1, 1],
        [2, 2, 2,  ..., 2, 1, 2],
        [1, 0, 0,  ..., 0, 0, 0]])

In [93]:
batch_size_actual = real_data_batch.size(0)
real_data_one_hot = one_hot_encode_sequences(real_data_batch, vocab_size)
real_data_one_hot = real_data_one_hot.view(batch_size_actual, -1)

In [96]:
real_data_one_hot

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 1.,  ..., 0., 1., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.]])

In [98]:
real_data_one_hot.shape

torch.Size([4, 24000])

In [100]:
with torch.no_grad():
    real_scores = critic(real_data_one_hot)
    generated_scores = critic(generated_data_tensor)

# Compare the scores
real_scores_mean = real_scores.mean().item()
generated_scores_mean = generated_scores.mean().item()

print(f"Mean Critic Score for Real Data: {real_scores_mean}")
print(f"Mean Critic Score for Generated Data: {generated_scores_mean}")

Mean Critic Score for Real Data: 1.394045114517212
Mean Critic Score for Generated Data: 1.1481131315231323
